In [ ]:
# Collegamento a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
lista=os.listdir('/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SPLIT_No_organize_CROP_CON_0+ INPAINTING_Normalized/TEST/masks')
print(len(lista))

563


In [ ]:
!pip install -q torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install -q mmcv==2.0.0rc4 -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html
!pip install -q mmsegmentation==1.2.2
!pip install -q mmengine==0.10.3
!pip install -q ftfy==6.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 738.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires torch>=1.13.0, but you have torch 1.11.0+cu113 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.3/452.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.11.0+cu113 True
1.2.2


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mmcv
import mmseg
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
working_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA'


# Definizione cartella contenente il dataset e classi
data_root = os.path.join(working_folder,'PolypGen-EIM-24-25_SPLIT_No_organize_CROP_CON_0+ INPAINTING_Normalized')
classes = ('background', 'object')
n_classes=len(classes)

# Calcolo del numero di immagini di training

# Supponiamo che le immagini di training siano contenute in una sottocartella chiamata "train" dentro data_root
train_folder = os.path.join(data_root, 'TRAIN/images')

# Conta il numero di file (immagini) nella cartella di training
train_length = len([f for f in os.listdir(train_folder) if os.path.isfile(os.path.join(train_folder, f))])

print(f"Numero di immagini di training: {train_length}")

Numero di immagini di training: 2631


REGISTRAZIONE DEL **DATASET**

In [ ]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################

# Classes and palette for VERSE dataset
classes = ('background', 'object')
palettePOLYP = [
    (0, 0, 0),       # background - black
    (255, 255, 255)  # object - white
]

@DATASETS.register_module()
class POLYP(BaseSegDataset):
    METAINFO = dict(classes=classes, palette=palettePOLYP)

    def __init__(self, **kwargs):
        super().__init__(img_suffix='.png',
                         seg_map_suffix='.png',
                         reduce_zero_label=False,
                         **kwargs)

3. Caricamento e visualizzazione del file di **configurazione**

In [ ]:
#Caricamento del config
from mmengine.config import Config
#caricamento della rete da mmsegmentation
config_file = os.path.join('/content/drive/MyDrive/CHALLENGE','mmseg_v1.2.2','mmseg_v1.2.2','configs','unet','unet-s5-d16_fcn_4xb4-160k_cityscapes-512x1024.py')
cfg = Config.fromfile(config_file)
cfg.device='cuda'

# Visualizzazione del config caricato (unet)
print(f'Config:\n{cfg.pretty_text}')

Config:
crop_size = (
    512,
    1024,
)
data_preprocessor = dict(
    bgr_to_rgb=True,
    mean=[
        123.675,
        116.28,
        103.53,
    ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        512,
        1024,
    ),
    std=[
        58.395,
        57.12,
        57.375,
    ],
    type='SegDataPreProcessor')
data_root = 'data/cityscapes/'
dataset_type = 'CityscapesDataset'
default_hooks = dict(
    checkpoint=dict(by_epoch=False, interval=16000, type='CheckpointHook'),
    logger=dict(interval=50, log_metric_by_epoch=False, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='SegVisualizationHook'))
default_scope = 'mmseg'
device = 'cuda'
env_cfg = dict(
    cudnn_benchmark=True,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
img_ratios = [
    0.5,
    0.75,
    1.0,
    1.25

In [ ]:
# MODIFICA DEL CONFING DELLA RETE

# Sostituzione di tutti i 'SyncBN' con 'BN'
cfg.model.auxiliary_head.norm_cfg = dict(requires_grad=True, type='BN')
cfg.model.backbone.norm_cfg = dict(requires_grad=True, type='BN')
cfg.model.decode_head.norm_cfg = dict(requires_grad=True, type='BN')
cfg.norm_cfg = dict(requires_grad=True, type='BN')

# Aggiornamento del numero di classi da segmentare
cfg.model.auxiliary_head.num_classes = 2
cfg.model.decode_head.num_classes = 2

# Modifica della dimensione delle immagini da processare
cfg.crop_size = (256,256)
cfg.data_preprocessor.size = (256,256)
cfg.model.data_preprocessor.size = (256,256)

# Modifica della funzione di loss (DiceLoss)
cfg.model.decode_head.loss_decode=[dict(loss_weight=1.0,type='DiceLoss', use_sigmoid=False)]
cfg.model.auxiliary_head.loss_decode=[dict(loss_weight=0.2,type='DiceLoss', use_sigmoid=False)]

# Aggiornamento dataset e definizione dei percorsi
cfg.dataset_type = 'POLYP'
cfg.data_root = data_root
cfg.img_ratios = [0.95,1.0,1.05]

# Pipeline options
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label=False),
    dict(
        type='RandomResize',
        scale=(256,256),
        ratio_range=(0.95, 1.05),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(256,256), cat_max_ratio=0.95),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='PackSegInputs')
]


cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256,256), keep_ratio=True),
    dict(type='LoadAnnotations', reduce_zero_label=False),
    dict(type='PackSegInputs')
]

batch_size = 8

# Definizione dataloader
cfg.train_dataloader.batch_size = batch_size
cfg.train_dataloader.dataset.type = 'POLYP'
cfg.train_dataloader.dataset.data_root = data_root
cfg.train_dataloader.dataset.data_prefix.img_path = os.path.join('TRAIN','images')
cfg.train_dataloader.dataset.data_prefix.seg_map_path = os.path.join('TRAIN','masks')
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline

cfg.val_dataloader.batch_size = batch_size
cfg.val_dataloader.dataset.type = 'POLYP'
cfg.val_dataloader.dataset.data_root = data_root
cfg.val_dataloader.dataset.data_prefix.img_path = os.path.join('VALID','images')
cfg.val_dataloader.dataset.data_prefix.seg_map_path = os.path.join('VALID','masks')
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline

cfg.test_dataloader.batch_size = 1
cfg.test_dataloader.dataset.type = 'POLYP'
cfg.test_dataloader.dataset.data_root = data_root
cfg.test_dataloader.dataset.data_prefix.img_path = os.path.join('TEST','images')
cfg.test_dataloader.dataset.data_prefix.seg_map_path = os.path.join('TEST','masks')
cfg.test_dataloader.dataset.pipeline = cfg.test_pipeline

# Parametri di training
num_epochs = 20
refresh_val = 1 # number of epochs to refresh validation metrics
flag_pretrained = True

iter_for_epoch = round(train_length/batch_size)
max_iters = num_epochs*iter_for_epoch
interval_val = round(refresh_val*iter_for_epoch)
cfg.train_cfg = dict(type='IterBasedTrainLoop', max_iters=max_iters, val_interval=interval_val)
cfg.val_cfg = dict(type='ValLoop') # Use the default validation loop.
cfg.test_cfg = dict(type='TestLoop') # Use the default test loop.
cfg.workflow = [('train', 1), ('val', 1)]

# Opzione per utilizzare rete pre-allenata
if flag_pretrained==True:
    cfg.load_from = os.path.join('/content/drive/MyDrive/CHALLENGE','checkpoints','fcn_unet_s5-d16_4x4_512x1024_160k_cityscapes_20211210_145204-6860854e.pth')
else:
    cfg.load_from = None

5. Creazione del runner ed avvio della fase di training

In [ ]:
# Preparazione al training
#nome della cartella che conterra i pesi
path_weights = 'weights_unet_dice_loss_cambio_parametri'

from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from mmengine.runner import Runner
from mmseg.registry import RUNNERS

from mmseg.apis import init_model
model = init_model(cfg)

cfg.visualizer.save_dir = os.path.join(working_folder,path_weights)
cfg.visualizer.classes = classes
cfg.visualizer.palette = [[0, 0, 0], [255, 255, 255]]
cfg.default_hooks.checkpoint.interval = interval_val

cfg.update(dict(work_dir=os.path.join(working_folder,path_weights)))
cfg.update(dict(launcher='none'))

runner = RUNNERS.build(cfg)

/usr/local/lib/python3.10/dist-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/usr/local/lib/python3.10/dist-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '


01/08 15:38:40 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 135997329
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.2, V12.2.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 1.11.0+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.5.2 (Git Hash a9302535553c73243c632ad3c4c80beec3d19a1e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-

/usr/local/lib/python3.10/dist-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/usr/local/lib/python3.10/dist-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/usr/local/lib/python3.10/dist-packages/mmseg/engine/hooks/visualization_hook.py:60: UserWarning: The draw is False, it means that the hook for visualization will not take effect. The results will NOT be visualized or stored.
  warnings.warn('The draw is False, it means that the '


In [ ]:
  # training
runner.train()

/usr/local/lib/python3.10/dist-packages/mmseg/datasets/transforms/loading.py:83: UserWarning: `reduce_zero_label` will be deprecated, if you would like to ignore the zero label, please set `reduce_zero_label=True` when dataset initialized
  warnings.warn('`reduce_zero_label` will be deprecated, '
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


01/08 15:38:48 - mmengine - WARNING - The prefix is not set in metric class IoUMetric.
Loads checkpoint by local backend from path: /content/drive/MyDrive/CHALLENGE/checkpoints/fcn_unet_s5-d16_4x4_512x1024_160k_cityscapes_20211210_145204-6860854e.pth
The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([19, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 64, 1, 1]).
size mismatch for decode_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for auxiliary_head.conv_seg.weight: copying a param with shape torch.Size([19, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 64, 1, 1]).
size mismatch for auxiliary_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
01/08 15:38:55 - mmengine - INFO 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


01/08 15:45:09 - mmengine - INFO - Iter(val) [50/71]    eta: 0:00:31  time: 1.6376  data_time: 1.4383  memory: 4928  
01/08 15:45:37 - mmengine - INFO - per class results:
01/08 15:45:37 - mmengine - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 95.35 | 99.32 |
|   object   | 26.79 | 29.84 |
+------------+-------+-------+
01/08 15:45:37 - mmengine - INFO - Iter(val) [71/71]    aAcc: 95.4300  mIoU: 61.0700  mAcc: 64.5800  data_time: 1.2629  time: 1.4509
01/08 15:45:49 - mmengine - INFO - Iter(train) [ 350/6580]  lr: 9.9806e-03  eta: 1:31:32  time: 0.5427  data_time: 0.0130  memory: 4928  loss: 0.0456  decode.loss_dice: 0.0376  decode.acc_seg: 88.6619  aux.loss_dice: 0.0079  aux.acc_seg: 90.5875
01/08 15:46:16 - mmengine - INFO - Iter(train) [ 400/6580]  lr: 9.9778e-03  eta: 1:26:35  time: 0.5515  data_time: 0.0124  memory: 4928  loss: 0.0431  decode.loss_dice: 0.0359  decode.acc_seg: 97.0541  aux.loss_dice: 0.0073  au

EncoderDecoder(
  (data_preprocessor): SegDataPreProcessor()
  (backbone): UNet(
    (encoder): ModuleList(
      (0): Sequential(
        (0): BasicConvBlock(
          (convs): Sequential(
            (0): ConvModule(
              (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (activate): ReLU(inplace=True)
            )
            (1): ConvModule(
              (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (activate): ReLU(inplace=True)
            )
          )
        )
      )
      (1): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): BasicConvBlock(
          (convs): Sequential(
            (0): ConvModule(
    